![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [22]:
# Stratégie Crypto Momentum « Risk-Averse »

Ce notebook présente une stratégie de trading visant à **réduire le risque** (drawdown, volatilité) tout en recherchant une **bonne rentabilité** et un **Sharpe Ratio** au-dessus de 1.  

## 1. Introduction

### 1.1 Contexte et Motivation

- Le marché des cryptomonnaies est connu pour sa **volatilité importante** et ses **grandes tendances** (haussières ou baissières).  
- L’objectif ici est de profiter du **momentum haussier** lorsque le marché est “bull”, tout en réduisant l’exposition lors des phases baissières.  
- Nous souhaitons **contrôler** le drawdown (sous 30 %) et viser un **Sharpe Ratio** > 1, indicateur qu’on a un rendement ajusté au risque satisfaisant.

### 1.2 Approche Générale

1. **Filtre de Marché** :  
   - Utiliser l’EMA 200 daily sur BTC pour déterminer si le marché global est haussier.  
   - Ajouter un check RSI daily sur BTC (ex. RSI > 55) pour renforcer ce filtre.
2. **Signal d’Entrée** :  
   - Indicateurs en *timeframe horaire* (RSI, MACD, Bollinger Bands).  
   - RSI > 50, MACD haussier, prix > bande supérieure de Bollinger → signaux de momentum haussier.  
3. **Sélection des Cryptos** :  
   - On veut éviter la sur-exposition : max 2 positions simultanées.  
   - On compare un ROC(14) daily (taux de variation) et ne prenons que celles qui ont un momentum positif (ROC>0) le plus élevé.  
4. **Position Sizing** (défensif) :  
   - 1 % du capital risqué par trade (`riskPerc = 0.01`).  
5. **Gestion du Risque** :  
   - *Trailing Stop* dynamique basé sur ATR.  
   - *Stop-Loss Global* à -25 %.  
   - *Sortie Partielle* rapide (par ex. +2 ATR).  

---

## 2. Mise en Place du Code

Ci-dessous, nous présentons une implémentation dans l’**environnement QuantConnect** (C# ou Python). Ici c’est du Python, en s’appuyant sur la librairie `AlgorithmImports`.

```python
# 2.1 Importations
from AlgorithmImports import *

class RiskAverseMomentumStrategy(QCAlgorithm):
    def Initialize(self):
        # Paramètres basiques du backtest
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2024, 12, 1)
        self.SetCash(100000)
        
        # Universe Crypto
        self.cryptoSymbols = ["BTCUSD", "ETHUSD", "LTCUSD"]
        self.symbols = []
        for ticker in self.cryptoSymbols:
            symbolObj = self.AddCrypto(ticker, Resolution.Hour)
            self.symbols.append(symbolObj.Symbol)
        
        # Benchmark
        self.SetBenchmark("BTCUSD")

        # BTC en daily pour filtrage bull/bear
        self.btcDaily = self.AddCrypto("BTCUSD", Resolution.Daily).Symbol
        self.dailyEma200 = self.EMA(self.btcDaily, 200, Resolution.Daily)
        self.dailyRsiBtc = self.RSI(self.btcDaily, 14, MovingAverageType.Wilders, Resolution.Daily)
        self.SetWarmUp(200, Resolution.Daily)
        
        # Création des indicateurs en hourly + ROC en daily pour scoring
        self.indicators = {}
        for sym in self.symbols:
            rsi = self.RSI(sym, 14, MovingAverageType.Wilders, Resolution.Hour)
            atr = self.ATR(sym, 14, MovingAverageType.Wilders, Resolution.Hour)
            macd = self.MACD(sym, 12, 26, 9, MovingAverageType.Wilders, Resolution.Hour)
            boll = self.BB(sym, 20, 2, MovingAverageType.Wilders, Resolution.Hour)
            
            # On ajoute un ROC daily comme critère de sélection
            dailySymbol = self.AddCrypto(sym.Value, Resolution.Daily).Symbol
            rocDaily = self.ROC(dailySymbol, 14, Resolution.Daily)

            self.indicators[sym] = {
                "rsi": rsi,
                "atr": atr,
                "macd": macd,
                "bollinger": boll,
                "rocDaily": rocDaily,
                "stop_price": None,
                "max_price": None,
                "entry_price": None,
                "partial_exit_done": False
            }
        
        # Paramètres de gestion du risque et money management
        self.riskPerc = 0.01               # 1% du capital risqué par trade
        self.trailingMultiplier = 1.5      # Trailing stop plus serré
        self.minHoldingPeriod = 48         # Minimum d'heures à conserver la position
        self.maxPortfolioDD = -0.25        # Stop global si drawdown -25%
        
        self.partialExitAtrMultiple = 2.0  # Sortie partielle à +2 ATR
        self.partialExitFraction = 0.5     # 50% liquidation partielle

        self.maxConcurrentPositions = 2    # Limiter à 2 positions
        self.positionsOpened = {}          # Track du moment d'ouverture
        
        # Équity initiale pour le drawdown global
        self.initialPortfolioValue = self.Portfolio.TotalPortfolioValue

    def OnData(self, data):
        # Skip si warming up
        if self.IsWarmingUp:
            return
        
        # 1) Stop-loss global si drawdown trop fort
        if (self.Portfolio.TotalPortfolioValue / self.initialPortfolioValue - 1) < self.maxPortfolioDD:
            self.Liquidate()
            return
        
        # 2) Filtre de marché global BTC Daily : EMA200 + RSI > 55
        btcDailyPrice = self.Securities[self.btcDaily].Close
        isBullMarket = (btcDailyPrice > self.dailyEma200.Current.Value) and (self.dailyRsiBtc.Current.Value > 55)
        
        if not isBullMarket:
            # Liquider si on passe en mode "bear"
            for sym in self.symbols:
                if self.Portfolio[sym].Invested:
                    self.Liquidate(sym)
                    self.resetPositionData(sym)
            return
        
        # 3) Scoring des cryptos via ROC daily
        scoring = []
        for sym in self.symbols:
            if self.indicators[sym]["rocDaily"].IsReady:
                roc_val = self.indicators[sym]["rocDaily"].Current.Value
                scoring.append((sym, roc_val))
        
        # On conserve celles qui ont ROC > 0 et on trie par score décroissant
        scoringPositive = [(s,sc) for (s,sc) in scoring if sc > 0]
        scoringPositive.sort(key=lambda x: x[1], reverse=True)
        bestSymbols = [s[0] for s in scoringPositive[:self.maxConcurrentPositions]]
        
        # 4) Recherche de signal d'entrée
        currentOpenPositions = sum(1 for sym in self.symbols if self.Portfolio[sym].Invested)
        
        for sym in bestSymbols:
            if currentOpenPositions >= self.maxConcurrentPositions:
                break
            
            if not data.ContainsKey(sym):
                continue
            
            ind = self.indicators[sym]
            if not all([ind["rsi"].IsReady, ind["macd"].IsReady, ind["bollinger"].IsReady, ind["atr"].IsReady]):
                continue
            
            price = self.Securities[sym].Price
            macd_hist = ind["macd"].Current.Value - ind["macd"].Signal.Current.Value
            
            # Conditions d'entrée : RSI>50, MACD haussier, prix > Bollinger sup
            if (ind["rsi"].Current.Value > 50 and 
                macd_hist > 0 and
                price > ind["bollinger"].UpperBand.Current.Value):
                
                if not self.Portfolio[sym].Invested:
                    # Calcul du ratio de position via ATR
                    atr_val = ind["atr"].Current.Value
                    stop_dist = self.trailingMultiplier * atr_val
                    capital = self.Portfolio.TotalPortfolioValue
                    capital_risk = capital * self.riskPerc
                    quantity = capital_risk / stop_dist
                    ratio = (quantity * price) / capital
                    
                    self.SetHoldings(sym, ratio)
                    
                    # Init variables de suivi
                    ind["stop_price"] = price - stop_dist
                    ind["max_price"]  = price
                    ind["entry_price"] = price
                    ind["partial_exit_done"] = False
                    self.positionsOpened[sym] = self.Time
                    currentOpenPositions += 1
        
        # 5) Gestion des positions ouvertes (Trailing stop, sortie partielle)
        for sym in self.symbols:
            if self.Portfolio[sym].Invested and data.ContainsKey(sym):
                ind = self.indicators[sym]
                price = data[sym].Close
                
                # MAJ max_price
                if ind["max_price"] is None or price > ind["max_price"]:
                    ind["max_price"] = price
                
                # Trailing stop dynamique
                atr_val = ind["atr"].Current.Value
                new_stop = ind["max_price"] - self.trailingMultiplier * atr_val
                if new_stop > ind["stop_price"]:
                    ind["stop_price"] = new_stop
                
                # Calcul holding period
                holding_hours = (self.Time - self.positionsOpened[sym]).total_seconds() / 3600
                
                # Sortie partielle si on atteint +2 ATR
                if not ind["partial_exit_done"]:
                    if price >= ind["entry_price"] + self.partialExitAtrMultiple * atr_val:
                        halfQty = 0.5 * self.Portfolio[sym].Quantity
                        self.MarketOrder(sym, -halfQty)
                        ind["partial_exit_done"] = True
                
                # Liquidation si stop atteint (après minHoldingPeriod)
                if holding_hours >= self.minHoldingPeriod:
                    if price < ind["stop_price"]:
                        self.Liquidate(sym)
                        self.resetPositionData(sym)
    
    def resetPositionData(self, sym):
        """ Remise à zéro des variables de position pour un symbole. """
        ind = self.indicators[sym]
        ind["stop_price"] = None
        ind["max_price"] = None
        ind["entry_price"] = None
        ind["partial_exit_done"] = False
        if sym in self.positionsOpened:
            del self.positionsOpened[sym]
